In [ ]:
!pip install git+https://github.com/rwalk/gsdmm.git
import pandas as pd
import os,re,gensim,nltk,spacy
from tqdm import tqdm
import numpy as np
from gsdmm import MovieGroupProcess
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel
from tabulate import tabulate
from warnings import filterwarnings
filterwarnings('ignore')
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-4b8qian_
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-4b8qian_
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
class PlymGSDMM : 
  def ApplyGSDMM(Path,File,Column,K = 3,AlphaStep=20,BetaStep=20) : 
    def DataCleaning(x,AllTextSet) : 
      x = ' '.join([str(i).lower().strip() for i in str(x).split()])
      x = ' '.join([i for i in x.split() if i in AllTextSet])
      return x
    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    def make_n_grams(texts):
        bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        trigram = gensim.models.Phrases(bigram[texts], threshold=100)
        trigram_mod = gensim.models.phrases.Phraser(trigram)
        bigrams_text = [bigram_mod[doc] for doc in texts]
        trigrams_text =  [trigram_mod[bigram_mod[doc]] for doc in bigrams_text]
        return trigrams_text      
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in gensim.parsing.preprocessing.STOPWORDS] for doc in texts]
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out      
    def top_words(mgp,cluster_word_distribution, top_cluster, values):
      Text = ''
      TheseResults = []
      for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        Text+= "\nCluster %s : %s"%(cluster,sort_dicts)
        TheseResults.append([cluster,sort_dicts])
      return Text,TheseResults  
    def EmbeddingValues(ThisWord) : 
      return [float(i) for i in EnglishWE[EnglishWE['Word'] == ThisWord]['WE'].values[0].strip("'").split(',')]


    ###########################################################
    data = pd.read_csv(os.path.join(Path,File))
    Phrases = data[Column].tolist()
    CustomizedStopWords =  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
                        'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have',
                        'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through','to', 'from', 'in',
                        'on', 'off', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'other', 'some', 'such','only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
                        'will', 'just', 'don', "don't", 'should', "should've",'&amp','amp','lia','li','ul','ol','blockquote','strong','table','font','image','someone','anyone','everyone','hello','hi','morning','can','use','evening','bye','b','em','mark','small','del','ins','sub','sup','tr']
    
    ThisFileWords = ' '.join([str(i).lower().strip() for i in Phrases])
    ThisFileWords = [re.sub('[^A-Za-z0-9]+', '',i) for i in ThisFileWords.split()]
    EnglishWE = pd.read_csv('/content/drive/MyDrive/Code/WE.csv')
    ValidEnglishWords = EnglishWE['Word'].tolist()
        
    OriginalWords = list(set(ThisFileWords))
    AllTextSet = list(set(ThisFileWords) & set(ValidEnglishWords))

    ThisFileWords = ' '.join([str(i).lower() for i in Phrases])
    AllTextSet = list(set(ThisFileWords.split()) & set(ValidEnglishWords))
    RemovedWords = [i for i in OriginalWords if not i in AllTextSet]
    AllTextSet = [str(i).lower() for i in AllTextSet if not i in RemovedWords]
    AllTextSet = [i for i in AllTextSet if not i in CustomizedStopWords]
    AllTextSet = [i for i in AllTextSet if  i.isalpha()]
    AllTextSet = [i for i in AllTextSet if  len(i) >1]
    
    
    WordDict = {'OriginalWords': OriginalWords,'CleanedWords' : AllTextSet,'RemovedWords' : RemovedWords}
    print(f'Number of OriginalWords is {len(WordDict["OriginalWords"])} , Number of CleanedWords is {len(WordDict["CleanedWords"])} , Number of RemovedWords is {len(WordDict["RemovedWords"])} so words exist with percentange {np.round(100*len(AllTextSet)/len(OriginalWords),2)}% ')
    data['CleanedText'] = data[Column].apply(lambda x : DataCleaning(x,AllTextSet))

    OriginalShape =  data.shape[0]
    data['WordLength'] = data['CleanedText'].apply(lambda x : len(str(x).split()))
    data = data[data['WordLength']>1]
    print(f'Dropped : {OriginalShape - data.shape[0]} rows')
    data.reset_index(inplace=True)
    EnglishWE = {ThisWord:EmbeddingValues(ThisWord) for ThisWord in AllTextSet}
    print(f'Number of words in WE dictionary {len(list(EnglishWE.keys()))}')


    global MainPath,SelectedData
    print(f'Applying for Cluster {K}')
    tokens_reviews = list(sent_to_words(data['CleanedText']))
    tokens_reviews = make_n_grams(tokens_reviews)
    reviews_lemmatized = lemmatization(tokens_reviews, allowed_postags=['NOUN', 'VERB', 'ADV'])
    reviews_lemmatized = remove_stopwords(reviews_lemmatized)
    print(reviews_lemmatized)
    np.random.seed(0)
    Results =[]
    X = 0
    
    for Alpha in list(np.linspace(0.05,1,AlphaStep)) :  # using different values for alpha
      for Beta in list(np.linspace(0.05,1,BetaStep)) : # using different values for beta
        X+=1
        print(f'Phase Number {X}')
        mgp = MovieGroupProcess(K=K, alpha=Alpha, beta=Beta, n_iters=30)
        vocab = set(x for review in reviews_lemmatized for x in review)
        n_terms = len(vocab)
        model = mgp.fit(reviews_lemmatized, n_terms)
        doc_count = np.array(mgp.cluster_doc_count)
        top_index = doc_count.argsort()[-10:][::-1]
        Loss =0
        for i in range(K) : 
          Loss += abs((1/K)-(doc_count[i]/sum(doc_count)))
        Results.append({'Parameters' : [Alpha,Beta],
                        'Loss':Loss,
                        'Doc Number':doc_count,
                        'Top Index':doc_count.argsort()[-10:][::-1],
                        'Top Words' : top_words(mgp,mgp.cluster_word_distribution, top_index, 10)[0],
                        'All Words' : top_words(mgp,mgp.cluster_word_distribution, top_index, 100)[1]})
    
    
    ResultsDF = pd.DataFrame(columns = ['Parameters', 'Loss', 'Doc Number', 'Top Index', 'Top Words', 'All Words'])
    for n,i in enumerate(Results) : 
      ResultsDF.loc[n] = list(i.values())
    ResultsDF  
    TotalCluster = [f'Cluster {i}' for i in range(K)]
    for j in TotalCluster : 
      ResultsDF[j] =ResultsDF['Top Words'].apply(lambda x : [i for i in str(x.strip('\n')).split('\n') if j in i][0].strip(f'{j} : ').strip('[]'))
    
    for j in range(K) : 
      ResultsDF[f'All Cluster {j}'] = ResultsDF['All Words'].apply(lambda x : [i[1] for i in x if i[0]==j])
    ResultsDF.to_csv(os.path.join(Path,f'Clusters Number {K}.csv'))  
    return None

#----------------------------------------------------
  def EvaluateGSDMM(Path,File,Method = 'All') : 
    SavedWords = {}
    def ClusterClean(x) : 
      x = str(x)
      if ',' in str(x) : 
        List = [re.sub('[^A-Za-z0-9,]+', '', i).split(',')  for i in x.split('),')]
        List = [[i[0],int(i[1])] for i in List]
        ThisList = []
        for l in range(0,len(List),2) : 
          if l < len(List)-1 : 
            ThisList.append((List[l],List[l+1]))
        return List
      else :
        return x
    def WordDifference(Word1,Word2,SavedWords) :
      
      Word1,Word2 = Word1.lower(),Word2.lower()
      if not  Word1 in SavedWords.keys() or not  Word2 in SavedWords.keys() : print('Word not found')
      ThisResult = np.linalg.norm(np.array(SavedWords[Word1])-np.array(SavedWords[Word2]), ord = 2) 
      return ThisResult   

    def ClustersCostFunctionAllWords(Clusters,SavedWords,ShowResults = True) : 
      global GetAllWords
      ClustersDiffs = []
      BetweenDiffs = []
      if ShowResults : print('Calculating Within Clusters Variance')
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        AllWords = Cl.split("'")[1::2]
        if len(AllWords) > 5 : 
          Diffs = []
          for i in range(len(AllWords)-1) : 
            for j in range(i+1,len(AllWords)) : 
              if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
                Diffs.append(float(WordDifference(AllWords[i],AllWords[j],SavedWords)))
              else : 
                Diffs.append(0)
          
      ClusterWords = []
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
      if ShowResults : print('Calculating Between Clusters Variance')
      for i in range(len(ClusterWords)) : 
        TheseWords = ClusterWords[i]
        ExList = [j for j in range(len(ClusterWords)) if j != i]
        OtherWords = []
        for k in ExList : 
          OtherWords.extend(ClusterWords[k])
        
        for Word1 in TheseWords : 
          for Word2 in OtherWords : 
            if Word1 in SavedWords.keys() and Word2 in SavedWords.keys() : 
              BetweenDiffs.append(float(WordDifference(Word1,Word2,SavedWords)))
            else : 
              BetweenDiffs.append(0)
      F1 = np.mean(BetweenDiffs)
      F2 = np.mean(ClustersDiffs)
      if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
      return F1,F2     

    def ClustersCostFunctionMean(Clusters,SavedWords,ShowResults = True) : 
      global GetAllWords 
      ClustersDiffs = []
      BetweenDiffs = []
      if ShowResults : print('Calculating Within Clusters Variance')
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        AllWords = Cl.split("'")[1::2]
        if len(AllWords) > 5 : 
          Diffs = []
          for i in range(len(AllWords)-1) : 
            for j in range(i+1,len(AllWords)) : 
              if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
                Diffs.append(float(WordDifference(AllWords[i],AllWords[j],SavedWords)))
              else : 
                Diffs.append(0)
          ClustersDiffs.append(np.mean(Diffs))
      ClusterWords = []
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
      if ShowResults : print('Calculating Between Clusters Variance')

      TheseWordsValuesMean = []
      for i in range(len(ClusterWords)) : 
        TheseWords = ClusterWords[i]
        for EachWord in TheseWords : 
          EachWord = EachWord.lower()
          if not EachWord in SavedWords.keys() : 
            if EachWord in EnglishWE.keys()  : 
              SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
            else : 
              SavedWords[EachWord] = [0 for i in range(300)]
        TheseWordsValues = [SavedWords[i] for i in TheseWords]
        if len(TheseWordsValues) > 0 : 
          TheseWordsValuesMean.append([np.matrix(TheseWordsValues).T[i].mean() for i in range(300)])
      
      for a in range(len(TheseWordsValuesMean)) : 
        for b in range(a+1,len(TheseWordsValuesMean)) : 
          BetweenDiffs.append(sum([(float(g)-float(m))**2 for g,m in zip(TheseWordsValuesMean[a],TheseWordsValuesMean[b])]))

      F1 = np.mean(BetweenDiffs)
      F2 = np.mean(ClustersDiffs)
      if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
      return F1,F2  

    def ClustersCostFunctionMin(Clusters,SavedWords,ShowResults = True) : 
      global GetAllWords 
      ClustersDiffs = []
      BetweenDiffs = []
      if ShowResults : print('Calculating Within Clusters Variance')
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        AllWords = Cl.split("'")[1::2]
        if len(AllWords) > 5 : 
          Diffs = []
          for i in range(len(AllWords)-1) : 
            for j in range(i+1,len(AllWords)) : 
              if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
                Diffs.append(float(WordDifference(AllWords[i],AllWords[j],SavedWords)))
              else : 
                Diffs.append(0)
          ClustersDiffs.append(np.mean(Diffs))
      ClusterWords = []
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
      if ShowResults : print('Calculating Between Clusters Variance')
      AllTheseWords = []
      for i in range(len(ClusterWords)) : 
        TheseWords = ClusterWords[i]
        for EachWord in TheseWords : 
          EachWord = EachWord.lower()
          if not EachWord in SavedWords.keys() : 
            if EachWord in EnglishWE.keys()  : 
              SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
            else : 
              SavedWords[EachWord] = [0 for i in range(300)]
        TheseWordsValues = [SavedWords[i] for i in TheseWords]
        AllTheseWords.append(TheseWordsValues)
      for EachGroup in AllTheseWords : 
        for OtherGroup in [Group for Group in AllTheseWords if Group != EachGroup ] : 
          GroupsWordsDiff = []
          for WordA in  EachGroup : 
            for WordB in OtherGroup : 
              if WordA != WordB : 
                GroupsWordsDiff.append(sum([(float(g)-float(m))**2 for g,m in zip(WordA,WordB)]))
          if len(GroupsWordsDiff) > 0 : BetweenDiffs.append(min(GroupsWordsDiff))

      F1 = np.mean(BetweenDiffs)
      F2 = np.mean(ClustersDiffs)
      if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
      return F1,F2    

    def ClustersCostFunctionMax(Clusters,SavedWords,ShowResults = True) : 
      global GetAllWords
      ClustersDiffs = []
      BetweenDiffs = []
      if ShowResults : print('Calculating Within Clusters Variance')
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        AllWords = Cl.split("'")[1::2]
        if len(AllWords) > 5 : 
          Diffs = []
          for i in range(len(AllWords)-1) : 
            for j in range(i+1,len(AllWords)) : 
              if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
                Diffs.append(float(WordDifference(AllWords[i],AllWords[j],SavedWords)))
              else : 
                Diffs.append(0)
          ClustersDiffs.append(np.mean(Diffs))
      ClusterWords = []
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
      if ShowResults : print('Calculating Between Clusters Variance')

      AllTheseWords = []
      for i in range(len(ClusterWords)) : 
        TheseWords = ClusterWords[i]
        for EachWord in TheseWords : 
          EachWord = EachWord.lower()
          if not EachWord in SavedWords.keys() : 
            if EachWord in EnglishWE.keys()  : 
              SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
            else : 
              SavedWords[EachWord] = [0 for i in range(300)]
        TheseWordsValues = [SavedWords[i] for i in TheseWords]
        AllTheseWords.append(TheseWordsValues)
      for EachGroup in AllTheseWords : 
        for OtherGroup in [Group for Group in AllTheseWords if Group != EachGroup ] : 
          GroupsWordsDiff = []
          for WordA in  EachGroup : 
            for WordB in OtherGroup : 
              if WordA != WordB : 
                GroupsWordsDiff.append(sum([(float(g)-float(m))**2 for g,m in zip(WordA,WordB)]))
          
          if len(GroupsWordsDiff) > 0 : BetweenDiffs.append(max(GroupsWordsDiff))
      F1 = np.mean(BetweenDiffs)
      F2 = np.mean(ClustersDiffs)
      if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
      return F1,F2    

    def ClustersCostFunctionCoherence(Clusters,SavedWords,ShowResults = True) : 
      ClusterWords = []
      for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
        if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
      if ShowResults : print('Calculating Between Clusters Variance')
      AllTheseDict = {}
      for i in range(len(ClusterWords)) : 
        TheseWords = ClusterWords[i]
        TheseWords = [k.lower().strip() for k in TheseWords]
        AllTheseDict[i] = TheseWords
      AllCoherence = []
      for a in range(len(list(AllTheseDict.keys()))) : 
        for b in range(a+1,len(list(AllTheseDict.keys()))) : 
          if len(AllTheseDict[a]) > 1 and len(AllTheseDict[b])> 1 : 
            topics = [AllTheseDict[a],AllTheseDict[b]]
            AllNeededWords = []
            AllNeededWords.extend(AllTheseDict[a])
            AllNeededWords.extend(AllTheseDict[b])
            gensim_dictionary = corpora.Dictionary()
            for i,w in enumerate(AllNeededWords) : 
              common_dictionary.token2id[w] = i
              gensim_dictionary.token2id[w] = i
            common_corpus = [gensim_dictionary.doc2bow(text) for text in topics]
            cm = CoherenceModel(topics=topics, corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass') 
            coherence = cm.get_coherence() 
            AllCoherence.append(float(coherence))


      if len(AllCoherence) > 0 : 
        return np.mean(AllCoherence)
      else : 
        return 0.0
    def EmbeddingValues(ThisWord) : 
      return [float(i) for i in EnglishWE[EnglishWE['Word'] == ThisWord]['WE'].values[0].strip("'").split(',')]


    ##################################################
    EnglishWE = pd.read_csv('/content/drive/MyDrive/Code/WE.csv')
    ValidEnglishWords = EnglishWE['Word'].tolist()
    GetAllWords = []
    ResultsData = pd.read_csv(os.path.join(Path,File))
    w = len(ResultsData['Top Index'].tolist()[0].strip('[]').split())    
    for i in range(w) : 
      ResultsData[f'Cluster {i}'] = ResultsData[f'Cluster {i}'].apply(lambda x :  ClusterClean(x))
    for i in ResultsData['Top Words'].tolist() : 
      for j in i.split('Cluster')[1:] : 
        GetAllWords.extend(j.split("'")[1::2]) 
    print(f'\nTotal Number of Unique Words {str(len(set(GetAllWords)))}')
    p = 0
    for i in set(GetAllWords) : 
      if i in ValidEnglishWords : 
        p+=1
    print(f'Number of Exist Words is {p} and Percentage of Words Exists in English Words {100*np.round(p/len(set(GetAllWords)),3)} %')
    SavedWords = {}
    for word in GetAllWords : 
      if word in ValidEnglishWords : 
        SavedWords[word] = EmbeddingValues(word)
    print(f'Number of Words in SavedWords {len(list(SavedWords.keys()))} ')

    ResultsData['Parameters'] = ResultsData['Parameters'].apply(lambda x : [float(i) for i in x.strip('[]').split(',')])
    ResultsData['Doc Number'] = ResultsData['Doc Number'].apply(lambda x : [int(i) for i in x.strip('[]').split()])
    ResultsData['Top Index'] = ResultsData['Top Index'].apply(lambda x : [int(i) for i in x.strip('[]').split()])
    Losses = ResultsData['Loss'].tolist()
    Losses.sort()
    for i in range(w) : 
      ResultsData[f'Cluster {i}'] = ResultsData[f'Cluster {i}'].apply(lambda x :  ClusterClean(x))
    Losses = ResultsData['Loss'].tolist()
    Losses.sort()

    SortedResults = []
    for Loss in Losses : 
      for i in range(ResultsData.shape[0]) :
        if Loss ==ResultsData['Loss'][i] : 
          SortedResults.append({k:v for k,v in zip(list(ResultsData.keys()),[ResultsData[j][i] for j in list(ResultsData.keys())])})

    for i in tqdm (range(len(SortedResults))) : 
      for Cl in SortedResults[i]['Top Words'].split('Cluster')[1:] : 
        AllWords = Cl.split("'")[1::2]

    CostFunctionResults = {}
    for i in tqdm (range(len(SortedResults))) : 
      ThisClusterValue = list(ClustersCostFunctionAllWords(SortedResults[i],SavedWords,ShowResults=False))
      CostFunctionResults[i] = {'AllWCV':ThisClusterValue[1]}    
      CostFunctionResults[i]['AllBCV'] = ThisClusterValue[0]
    if Method in ['All','Mean'] : 
      for i in tqdm (range(len(SortedResults))) : 
        ThisClusterValue = list(ClustersCostFunctionMean(SortedResults[i],SavedWords,ShowResults=False))
        CostFunctionResults[i]['MeanBCV'] = ThisClusterValue[0]
        CostFunctionResults[i]['MeanWCV'] = ThisClusterValue[1]
    if Method in ['All','Min'] : 
      for i in tqdm (range(len(SortedResults))) : 
        ThisClusterValue = list(ClustersCostFunctionMin(SortedResults[i],SavedWords,ShowResults=False))
        CostFunctionResults[i]['MinBCV'] = ThisClusterValue[0]
        CostFunctionResults[i]['MinWCV'] = ThisClusterValue[1]
    if Method in ['All','Max'] : 
      for i in tqdm (range(len(SortedResults))) : 
        ThisClusterValue = list(ClustersCostFunctionMax(SortedResults[i],SavedWords,ShowResults=False))
        CostFunctionResults[i]['MaxBCV'] = ThisClusterValue[0]
        CostFunctionResults[i]['MaxWCV'] = ThisClusterValue[1]
    if Method in ['All','Coh'] : 
      for i in tqdm (range(len(SortedResults))) : 
        
        CostFunctionResults[i]['CohBCV'] = ClustersCostFunctionCoherence(SortedResults[i],SavedWords,ShowResults=False)

    for i in tqdm(CostFunctionResults.keys()): 
      AllWCV =  CostFunctionResults[i]['AllWCV']
      AllBCV =  CostFunctionResults[i]['AllBCV']
      if Method in ['All','Mean'] : MeanBCV =  CostFunctionResults[i]['MeanBCV']
      if Method in ['All','Min'] : MinBCV =  CostFunctionResults[i]['MinBCV']
      if Method in ['All','Max'] : MaxBCV =  CostFunctionResults[i]['MaxBCV']    
      if Method in ['All','Coh'] : CohBCV = CostFunctionResults[i]['CohBCV']                 

      ThisAllD,ThisMeanD,ThisMinD,ThisMaxD,ThisCohD = {},{},{},{},{}
      l=0
      for a in np.linspace(0.01, 1, num=5) : 
        for b in np.linspace(0.01, 1, num=5): 
          l+=1
          ThisAllD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(AllBCV))]
          if Method in ['All','Mean'] : ThisMeanD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MeanBCV))]
          if Method in ['All','Min'] : ThisMinD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MinBCV))]
          if Method in ['All','Max'] : ThisMaxD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MaxBCV))]
          if Method in ['All','Coh'] : ThisCohD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(CohBCV))]
      CostFunctionResults[i]['AllCF'] = ThisAllD
      CostFunctionResults[i]['MeanCF'] = ThisMeanD
      CostFunctionResults[i]['MinCF'] = ThisMinD
      CostFunctionResults[i]['MaxCF'] = ThisMaxD
      CostFunctionResults[i]['CohCF'] = ThisCohD
    for i in range(len(SortedResults)) : 
      for Item in ['AllBCV','AllWCV','MeanBCV','MinWCV','MaxBCV','MeanWCV','MinBCV','MaxWCV','CohBCV','AllCF','MeanCF','MinCF','MaxCF','CohCF'] : 
        if Item in CostFunctionResults[i].keys() : SortedResults[i][Item] = CostFunctionResults[i][Item]

    Alpha,Beta,A,B,AllBCV,MeanBCV,MinBCV,MaxBCV,CohBCV,WCV,IDs,AllCF,MeanCF,MinCF,MaxCF,CohCF = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    n=0
    for i in SortedResults : 
      for k in i['MeanCF'].values() : 
        MeanCF.append(k[2])
      for k in i['MinCF'].values() : 
        MinCF.append(k[2])
      for k in i['MaxCF'].values() : 
        MaxCF.append(k[2])      
      for k in i['CohCF'].values() : 
        CohCF.append(k[2])      
      for j,k in i['AllCF'].items() : 
        n+=1
        IDs.append(n)
        AllCF.append(k[2])
        AllBCV.append(i['AllBCV'])
        if Method in ['All','Mean'] :
          MeanBCV.append(i['MeanBCV'])
        else : 
          MeanBCV.append('-')
        if Method in ['All','Min'] :
          MinBCV.append(i['MinBCV'])
        else : 
          MinBCV.append('-')
        if Method in ['All','Max'] :
          MaxBCV.append(i['MaxBCV'])
        else : 
          MaxBCV.append('-')
        if Method in ['All','Coh'] :
          CohBCV.append(i['CohBCV'])       
        else : 
          CohBCV.append('-')       
        WCV.append(i['AllWCV'])
        A.append(k[0])
        B.append(k[1])
        Alpha.append(i['Parameters'][0])
        Beta.append(i['Parameters'][1])

    K = [IDs,Alpha,Beta,A,B,WCV,AllBCV,AllCF,MeanBCV,MeanCF,MinBCV,MinCF,MaxBCV,MaxCF,CohBCV,CohCF]
    L = [i if len(i)!=0 else ['-']*(len(IDs)) for i in K]
    ResultsDF =pd.DataFrame(L,index = ['IDs','Alpha','Beta','A','B','WCV','AllBCV','AllCF','MeanBCV','MeanCF','MinBCV','MinCF','MaxBCV','MaxCF','CohBCV','CohCF']).T
    ResultsDF.to_csv(os.path.join(Path,f'ResultsDF_{Method}_{w}.csv'))

    ThisFile = os.path.join(Path,f'ResultsDF_{Method}_{w}.csv')
    ResultsDF = pd.read_csv(ThisFile) 
    if 'index' in ResultsDF.columns : ResultsDF.drop(['index'],axis=1,inplace=True)
    if 'Unnamed: 0' in ResultsDF.columns : ResultsDF.drop(['Unnamed: 0'],axis=1,inplace=True)
    CalculatedCFs = ['AllCF','MeanCF','MinCF','MaxCF','CohCF']
    CFsDataFrame = pd.DataFrame()
    for CF in CalculatedCFs : 
      ResultLists = []
      for A in ResultsDF['A'].unique() : 
        for B in ResultsDF['B'].unique() : 
          MinCF = ResultsDF[ResultsDF['A'] == A][ResultsDF['B'] == B][CF].min()
          if ResultsDF[ResultsDF[CF] == MinCF].shape[0] > 1 : 
            print(ResultsDF[ResultsDF[CF] == MinCF])
            ThisList = ResultsDF[ResultsDF[CF] == MinCF].iloc[0,:].tolist()
            ResultLists.append([ThisList[0]] + ThisList[3:5]+ThisList[1:3]+ThisList[5:])
      col_names = ['Trial Number','A','B','Alpha','Beta','WCV','AllBCV','AllCF','MeanBCV','MeanCF','MinBCV','MinCF','MaxBCV','MaxCF','CohBCV','CohCF']
      
      ThisCFsDataFrame = pd.DataFrame(ResultLists, columns=col_names)
      ThisCFsDataFrame['SelectedCF'] = CF
      print(tabulate(ResultLists, headers=col_names, tablefmt="fancy_grid"))
      CFsDataFrame = pd.concat([CFsDataFrame,ThisCFsDataFrame],axis=0)
    CFsDataFrame.to_csv(os.path.join(Path,f'BestCFValue_{Method}_{w}.csv'))


    ThisFile = os.path.join(Path,f'ResultsDF_{Method}_{w}.csv')
    data = pd.read_csv(ThisFile)  
    print(f'for clutser {w} shape is  {data.shape}')
    for alpha in data['Alpha'].unique() : 
      for beta in data['Beta'].unique() : 
        if len(data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()) !=1 : print('error')
        if len(data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()) !=1 : print('error')
        if len(data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()) !=1 : print('error')
        if len(data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()) !=1 : print('error')
        if len(data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()) !=1 : print('error')

    AllBCVDict = {}
    n=0
    for alpha in data['Alpha'].unique() : 
      for beta in data['Beta'].unique() : 
        n+=1
        AllBCVDict[n] = {'Alpha': alpha,
                        'Beta':beta,
                        'AllBCV':data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()[0],
                        'WBCV':data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0],
                        'EqualParameters_AllBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()[0]}


        if Method in ['All','Mean'] :
          AllBCVDict[n]['MeanBCV'] = data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()[0]
          AllBCVDict[n]['EqualParameters_MeanBCV'] =  data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()[0]
        else : 
          AllBCVDict[n]['MeanBCV'] = 999999
          AllBCVDict[n]['EqualParameters_MeanBCV'] =  999999

        if Method in ['All','Min'] :
          AllBCVDict[n]['MinBCV'] = data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()[0]
          AllBCVDict[n]['EqualParameters_MinBCV'] =  data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()[0]
        else : 
          AllBCVDict[n]['MinBCV'] = 999999
          AllBCVDict[n]['EqualParameters_MinBCV'] =  999999          

        if Method in ['All','Max'] :
          AllBCVDict[n]['MaxBCV'] = data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()[0]
          AllBCVDict[n]['EqualParameters_MaxBCV'] =  data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()[0]
        else : 
          AllBCVDict[n]['MaxBCV'] = 999999
          AllBCVDict[n]['EqualParameters_MaxBCV'] =  999999          

        if Method in ['All','Coh'] :
          AllBCVDict[n]['CohBCV'] = data[data['Alpha']==alpha][data['Beta']==beta]['CohBCV'].unique()[0]
          AllBCVDict[n]['EqualParameters_CohBCV'] =  data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['CohBCV'].unique()[0]
        else : 
          AllBCVDict[n]['CohBCV'] = 999999
          AllBCVDict[n]['EqualParameters_CohBCV'] =  999999          

    data = pd.DataFrame(AllBCVDict).T
    data.fillna(99,inplace=True)
    data['MincF'] = data.apply(lambda x : min(x.AllBCV,x.MeanBCV,x.MinBCV,x.MaxBCV,x.WBCV,x.CohBCV,x.EqualParameters_AllBCV,x.EqualParameters_MeanBCV,x.EqualParameters_MinBCV,x.EqualParameters_MaxBCV,x.EqualParameters_CohBCV),axis=1)
    data = data.sort_values(['Alpha', 'Beta'],ascending = [True, True])
    data.to_csv(os.path.join(Path,f'CombinedCFs_{Method}_{w}.csv'))
    return CFsDataFrame



G = PlymGSDMM 
# G.ApplyGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'BT Forums Community.csv',Column = 'Description',K = 4,AlphaStep=10,BetaStep=10) 
G.EvaluateGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'Clusters Number 4.csv') 
G.EvaluateGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'Clusters Number 4.csv',Method = 'Mean') 
G.EvaluateGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'Clusters Number 4.csv',Method = 'Min') 
G.EvaluateGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'Clusters Number 4.csv',Method = 'Max') 
G.EvaluateGSDMM(Path = '/content/drive/MyDrive/Code/',File = 'Clusters Number 4.csv',Method = 'Coh') 


Total Number of Unique Words 101
Number of Exist Words is 101 and Percentage of Words Exists in English Words 100.0 %
Number of Words in SavedWords 101 


100%|██████████| 126/126 [00:00<00:00, 1374.37it/s]


╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞════════════════╪═════╪═════╪═════════╪════════╪═══════╪══════════╪═════════╪═══════════╪══════════╪══════════╪═════════╪══════════╪═════════╪══════════╪═════════╡
╘════════════════╧═════╧═════╧═════════╧════════╧═══════╧══════════╧═════════╧═══════════╧══════════╧══════════╧═════════╧══════════╧═════════╧══════════╧═════════╛
╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞═════════

100%|██████████| 126/126 [00:00<00:00, 2089.90it/s]


╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞════════════════╪═════╪═════╪═════════╪════════╪═══════╪══════════╪═════════╪═══════════╪══════════╪══════════╪═════════╪══════════╪═════════╪══════════╪═════════╡
╘════════════════╧═════╧═════╧═════════╧════════╧═══════╧══════════╧═════════╧═══════════╧══════════╧══════════╧═════════╧══════════╧═════════╧══════════╧═════════╛
╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞═════════

...

100%|██████████| 126/126 [00:00<00:00, 2968.82it/s]


╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞════════════════╪═════╪═════╪═════════╪════════╪═══════╪══════════╪═════════╪═══════════╪══════════╪══════════╪═════════╪══════════╪═════════╪══════════╪═════════╡
╘════════════════╧═════╧═════╧═════════╧════════╧═══════╧══════════╧═════════╧═══════════╧══════════╧══════════╧═════════╧══════════╧═════════╧══════════╧═════════╛
       IDs     Alpha      Beta     A       B  WCV    AllBCV  AllCF MeanBCV  \
0        1  0.261111  0.155556  0.01  0.0100  NaN  1.758215    NaN       -   
1        2  0.261111  0.155556  0.01  0.2575  NaN  1.758215    NaN       -   
2        3  0.261111  0.155556  0.01  0.5050  NaN  1.758215    NaN       -   
3        4  0.261111  0.1555

100%|██████████| 126/126 [00:00<00:00, 2311.16it/s]


╒════════════════╤═════╤═════╤═════════╤════════╤═══════╤══════════╤═════════╤═══════════╤══════════╤══════════╤═════════╤══════════╤═════════╤══════════╤═════════╕
│ Trial Number   │ A   │ B   │ Alpha   │ Beta   │ WCV   │ AllBCV   │ AllCF   │ MeanBCV   │ MeanCF   │ MinBCV   │ MinCF   │ MaxBCV   │ MaxCF   │ CohBCV   │ CohCF   │
╞════════════════╪═════╪═════╪═════════╪════════╪═══════╪══════════╪═════════╪═══════════╪══════════╪══════════╪═════════╪══════════╪═════════╪══════════╪═════════╡
╘════════════════╧═════╧═════╧═════════╧════════╧═══════╧══════════╧═════════╧═══════════╧══════════╧══════════╧═════════╧══════════╧═════════╧══════════╧═════════╛
       IDs     Alpha      Beta     A       B  WCV    AllBCV  AllCF MeanBCV  \
0        1  0.261111  0.155556  0.01  0.0100  NaN  1.758215    NaN       -   
1        2  0.261111  0.155556  0.01  0.2575  NaN  1.758215    NaN       -   
2        3  0.261111  0.155556  0.01  0.5050  NaN  1.758215    NaN       -   
3        4  0.261111  0.1555

,Trial Number,A,B,Alpha,Beta,WCV,AllBCV,AllCF,MeanBCV,MeanCF,MinBCV,MinCF,MaxBCV,MaxCF,CohBCV,CohCF,SelectedCF
0,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MeanCF
1,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MeanCF
2,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MeanCF
3,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MeanCF
4,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MeanCF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MaxCF
21,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MaxCF
22,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MaxCF
23,1,0.01,0.01,0.261111,0.155556,NaN,1.758215,NaN,-,-,-,-,-,-,-0.082151,NaN,MaxCF
